In [1]:
import os
import librosa
import soundfile as sf
from pydub import AudioSegment
from pydub.silence import split_on_silence
from scipy.signal import medfilt

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pathlib
import librosa

In [2]:
def split_audio_text(data_folder, output_folder, max_files=None):
    # Создаем каталог для сохранения файлов
    os.makedirs(output_folder, exist_ok=True)

    count_processed_files = 0

    for root, dirs, files in os.walk(data_folder):
        for file in files:
            if file.endswith(".txt"):
                text_path = os.path.join(root, file)
                audio_path = text_path.replace(".txt", ".wav")

                # Извлекаем текст
                with open(text_path, 'r', encoding='utf-8') as text_file:
                    text = text_file.read().strip()

                # Извлекаем аудио и сохраняем его
                audio_signal, sr = librosa.load(audio_path, sr=None)

                # Разделяем аудио на фрагменты
                audio_segments = split_audio(audio_signal)

                # Создаем подкаталоги для каждого слова
                for word in text.split():
                    word_folder = os.path.join(output_folder, word)
                    os.makedirs(word_folder, exist_ok=True)

                # Сохраняем фрагменты аудио в соответствующие папки
                for i, segment in enumerate(audio_segments):
                    word_folder = os.path.join(output_folder, text.split()[i])
                    word_audio_path = os.path.join(word_folder, f"{os.path.splitext(file)[0]}_{i+1}.wav")
                    segment.export(word_audio_path, format="wav")

                    # Сохраняем текст
                    word_text_path = os.path.join(word_folder, f"{os.path.splitext(file)[0]}_{i+1}.txt")
                    with open(word_text_path, 'w', encoding='utf-8') as word_text_file:
                        word_text_file.write(text.split()[i])

                count_processed_files += 1
                if max_files is not None and count_processed_files >= max_files:
                    return

In [3]:
def split_audio(audio_signal):
    # Преобразуем в AudioSegment
    audio_segment = AudioSegment(audio_signal.tobytes(), frame_rate=16000, sample_width=2, channels=1)

    # Удаляем тишину и усиливаем сигнал
    non_silent_audio = remove_silence(audio_segment)
    amplified_audio = non_silent_audio + 10  # Может потребоваться настройка уровня усиления

    # Разделяем аудио на фрагменты в зависимости от громкости
    silence_threshold = amplified_audio.dBFS - 16
    audio_segments = split_on_silence(amplified_audio, silence_thresh=silence_threshold, keep_silence=100)

    return audio_segments

In [4]:
def remove_silence(audio_segment):
    # Удаляем тишину с использованием pydub
    non_silent_audio = AudioSegment.silent(duration=1)
    non_silent_audio += audio_segment
    non_silent_audio = non_silent_audio.strip_silence(silence_len=100, silence_thresh=audio_segment.dBFS - 16)
    return non_silent_audio

In [5]:
# Замените пути к папкам на ваши
data_folder_path = "/home/redalexdad/recognition_speech/new_buriy_audiobooks_2_val"
output_folder_path = "/home/redalexdad/recognition_speech/voice_split"

In [6]:
split_audio_text(data_folder_path, output_folder_path, max_files=20)

In [6]:
data_dir = '/home/redalexdad/recognition_speech/voice_split'

# label_names = []
file_counter = 0  # Переменная для отслеживания количества обработанных файлов

# Рекурсивно обходим все файлы и подкаталоги в корневом каталоге
for root, dirs, files in os.walk(data_dir):
    for file in files:
        # Проверяем, что файл - звуковой файл
        if file.endswith(".wav"):
            # Увеличиваем счетчик обработанных файлов
            file_counter += 1

            # Проверяем, не превысило ли количество файлов 10
            if file_counter > 10:
                break

            # Остальной код вашего скрипта остается без изменений
            audio_file_path = os.path.join(root, file)
            label = os.path.basename(root)
            text_file_path = os.path.join(root, f"{os.path.splitext(file)[0]}.txt")

            if not os.path.exists(text_file_path):
                print(f"No text file found for {audio_file_path}")
                continue

            # Далее ваш код для чтения текста и вывода информации о звуке
            print(f"Label {label}")
            waveform, sample_rate = librosa.load(audio_file_path, sr=None)

            print("Audio shape:", waveform.shape)
            print('Waveform shape:', waveform.shape)
            # display(waveform)
            print('Audio playback')
            display.display(display.Audio(waveform, rate=sample_rate))

            plt.figure(figsize=(16, 3))
            plt.plot(waveform)
            plt.title(label)
            plt.yticks(np.arange(-1.2, 1.2, 0.2))
            plt.ylim([-1.1, 1.1])
            # Отображаем график после обработки
            plt.show()